In [1]:
import bioMass.dataViz as dv
from bioMass.dataloader import SentinelDataset, SentinelDataModule
from bioMass.model import S2_Unet
from pytorch_lightning import Trainer
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader, random_split
from tqdm.notebook import tqdm
from torch import nn
import os
import numpy as np
import pandas as pd

from pytorch_lightning import loggers as pl_loggers

%config Completer.use_jedi = False
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
tensorboard_logdir = '/home/ubuntu/Thesis/backup_data/bioMass_data/lightning_logs_pilot/'

logger = pl_loggers.TensorBoardLogger(
    tensorboard_logdir, name="", default_hp_metric=False
)

# trainer = Trainer(logger=logger, accelerator='gpu', max_epochs=20, devices=1, log_every_n_steps=1)
trainer = Trainer(logger=logger, accelerator='gpu', max_epochs=1000, devices=1, log_every_n_steps=1, overfit_batches=1)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(overfit_batches=1)` was configured so 1 batch will be used.


In [ ]:
loss_module = nn.MSELoss(reduction='mean')

dm = SentinelDataModule(max_chips=100, loader_device='cpu', num_workers=1)
model = S2_Unet(loss_fn=loss_module, lr=1e-4, init_n_features=64, in_channels=11, dropout_p=0.5)

trainer.fit(model, datamodule=dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

   | Name    | Type       | Params
----------------------------------------
0  | loss_fn | MSELoss    | 0     
1  | maxpool | MaxPool2d  | 0     
2  | up      | Upsample   | 0     
3  | EB0     | Sequential | 43.3 K
4  | EB1     | Sequential | 221 K 
5  | EB2     | Sequential | 885 K 
6  | EB3     | Sequential | 3.5 M 
7  | Bottom  | Sequential | 4.7 M 
8  | DB2     | Sequential | 2.4 M 
9  | DB1     | Sequential | 589 K 
10 | DB0     | Sequential | 147 K 
11 | out     | Sequential | 73.9 K
----------------------------------------
12.6 M    Trainable params
0         Non-trainable params
12.6 M    Total params
50.322    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/ubuntu/Thesis/miniconda3/envs/bioMass/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/ubuntu/Thesis/miniconda3/envs/bioMass/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:468: UserWarning: You requested to overfit but enabled train dataloader shuffling. We are turning off the train dataloader shuffling for you.
  rank_zero_warn(
/home/ubuntu/Thesis/miniconda3/envs/bioMass/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers`

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]